In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_curve, precision_recall_curve
from sklearn.metrics import roc_auc_score, average_precision_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.calibration import CalibratedClassifierCV
from collections import Counter
import warnings
import random
import joblib
import json
from datetime import datetime
warnings.filterwarnings('ignore')

# =====================================================
# CRITICAL: SET ALL RANDOM SEEDS FOR REPRODUCIBILITY
# =====================================================
RANDOM_STATE = 42

# Set all random seeds to ensure reproducible results
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

print("\nTARGETED HIGH-RISK CLASS PERFORMANCE IMPROVEMENT")
print("=" * 60)
print("Focus: Boosting Precision, Recall, and F1 for High-Risk Nutritional Patients")

# =====================================================
# STEP 1: ANALYZE CURRENT IMBALANCE PROBLEM
# =====================================================

print(f"\n1. ANALYZING CLASS IMBALANCE PROBLEM")
print("=" * 50)

# Load data
try:
    df_clustered = pd.read_csv('Binary_RF_SVD_clustering_results.csv')
    print(f"Loaded data: {df_clustered.shape}")
except FileNotFoundError:
    print("Please ensure SVD clustering results are available")
    exit()

# Prepare features and target
svd_cols = [col for col in df_clustered.columns if col.startswith('SVD_Component')]
df_modeling = df_clustered.copy()

feature_cols = ['Gender', 'Age'] + svd_cols
X = df_modeling[feature_cols].fillna(0)
y = (df_modeling['MUSTScore'] >= 2).astype(int)

# Analyze class distribution
class_counts = Counter(y)
total_samples = len(y)
high_risk_count = class_counts[1]
low_risk_count = class_counts[0]
imbalance_ratio = low_risk_count / high_risk_count

print(f"Class Distribution Analysis:")
print(f"  Low Risk (0):  {low_risk_count} samples ({low_risk_count/total_samples:.1%})")
print(f"  High Risk (1): {high_risk_count} samples ({high_risk_count/total_samples:.1%})")
print(f"  Imbalance Ratio: {imbalance_ratio:.1f}:1 (Low:High)")

if imbalance_ratio > 5:
    print(f"  SEVERE IMBALANCE DETECTED - Need aggressive techniques")
elif imbalance_ratio > 3:
    print(f"  MODERATE IMBALANCE - Standard techniques should work")
else:
    print(f"  MILD IMBALANCE - Basic class weighting sufficient")

# =====================================================
# STEP 2: STRATEGIC OVERSAMPLING WITH VARIATION
# =====================================================

print(f"\n2. STRATEGIC OVERSAMPLING WITH SMART VARIATION")
print("=" * 50)

def create_smart_synthetic_samples(X_minority, y_minority, n_synthetic=50, noise_level=0.1, random_state=None):
    """Create synthetic samples with small variations from existing high-risk patients"""
    
    # Set local random state for this function
    if random_state is not None:
        local_rng = np.random.RandomState(random_state)
    else:
        local_rng = np.random.RandomState(RANDOM_STATE + 100)  # Offset to avoid conflicts
    
    synthetic_samples = []
    synthetic_labels = []
    
    # Convert to numpy for easier manipulation
    X_min_array = X_minority.values
    
    for i in range(n_synthetic):
        # Use local_rng for reproducibility
        base_idx = local_rng.randint(0, len(X_min_array))
        base_sample = X_min_array[base_idx].copy()
        
        # Add small random variations
        for j in range(len(base_sample)):
            if local_rng.random() < 0.3:  # 30% chance to modify each feature
                if j < 2:  # Demographic features (Gender, Age)
                    if j == 0:  # Gender - keep unchanged
                        continue
                    else:  # Age - small variation
                        base_sample[j] += local_rng.normal(0, 2)  # +/- 2 years
                else:  # SVD components - small noise
                    base_sample[j] += local_rng.normal(0, abs(base_sample[j]) * noise_level)
        
        synthetic_samples.append(base_sample)
        synthetic_labels.append(1)  # All are high-risk
    
    # Convert back to DataFrame
    synthetic_df = pd.DataFrame(synthetic_samples, columns=X_minority.columns)
    
    return synthetic_df, np.array(synthetic_labels)

def create_balanced_dataset(X, y, strategy='moderate'):
    """Create balanced dataset with different intensity levels"""
    
    # Separate classes
    X_low_risk = X[y == 0]
    X_high_risk = X[y == 1]
    y_low_risk = y[y == 0]
    y_high_risk = y[y == 1]
    
    print(f"Original: {len(X_low_risk)} low-risk, {len(X_high_risk)} high-risk")
    
    if strategy == 'conservative':
        # Increase high-risk by 2x
        target_high_risk = len(X_high_risk) * 2
    elif strategy == 'moderate':
        # Increase high-risk to 1:2 ratio (instead of 1:7)
        target_high_risk = len(X_low_risk) // 2
    elif strategy == 'aggressive':
        # Near balance: 1:1.5 ratio
        target_high_risk = int(len(X_low_risk) * 0.67)
    else:
        target_high_risk = len(X_high_risk)
    
    # Calculate how many synthetic samples needed
    n_synthetic = max(0, target_high_risk - len(X_high_risk))
    
    if n_synthetic > 0:
        print(f"Creating {n_synthetic} synthetic high-risk samples...")
        
        # Create synthetic samples with reproducible random state
        strategy_seed = RANDOM_STATE + {'conservative': 10, 'moderate': 20, 'aggressive': 30}.get(strategy, 0)
        X_synthetic, y_synthetic = create_smart_synthetic_samples(
            X_high_risk, y_high_risk, n_synthetic, random_state=strategy_seed
        )
        
        # Combine all data
        X_balanced = pd.concat([X_low_risk, X_high_risk, X_synthetic], ignore_index=True)
        y_balanced = np.concatenate([y_low_risk, y_high_risk, y_synthetic])
        
        print(f"Balanced: {len(X_low_risk)} low-risk, {len(X_high_risk) + n_synthetic} high-risk")
        
    else:
        X_balanced, y_balanced = X, y
        print(f"No oversampling needed for {strategy} strategy")
    
    return X_balanced, y_balanced

# Create different balanced datasets
strategies = ['conservative', 'moderate', 'aggressive']
balanced_datasets = {}

for strategy in strategies:
    X_bal, y_bal = create_balanced_dataset(X, y, strategy)
    balanced_datasets[strategy] = (X_bal, y_bal)

# =====================================================
# STEP 3: HIGH-RECALL OPTIMIZED MODELS
# =====================================================

print(f"\n3. HIGH-RECALL OPTIMIZED MODELS")
print("=" * 50)

def evaluate_high_risk_focused_model(X_train, X_test, y_train, y_test, model, model_name):
    """Evaluate model with focus on high-risk class performance"""
    
    # Train model
    model.fit(X_train, y_train)
    
    # Get predictions and probabilities
    y_pred = model.predict(X_test)
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_pred_proba = model.decision_function(X_test)
    
    # Standard metrics
    report = classification_report(y_test, y_pred, target_names=['Low Risk', 'High Risk'], 
                                 output_dict=True, zero_division=0)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # High-risk focused metrics
    high_risk_precision = report['High Risk']['precision']
    high_risk_recall = report['High Risk']['recall']
    high_risk_f1 = report['High Risk']['f1-score']
    
    print(f"\n{model_name} Results:")
    print(f"  AUC: {auc:.3f}")
    print(f"  High-Risk Precision: {high_risk_precision:.3f}")
    print(f"  High-Risk Recall:    {high_risk_recall:.3f}")
    print(f"  High-Risk F1:        {high_risk_f1:.3f}")
    
    return {
        'model': model,
        'auc': auc,
        'high_risk_precision': high_risk_precision,
        'high_risk_recall': high_risk_recall,
        'high_risk_f1': high_risk_f1,
        'y_pred_proba': y_pred_proba,
        'classification_report': report
    }

def test_optimized_algorithms(X, y, test_size=0.3):
    """Test algorithms specifically optimized for high-risk detection"""
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=RANDOM_STATE, stratify=y
    )
    
    results = {}
    
    # 1. Highly Weighted Logistic Regression
    print("Testing High-Weight Logistic Regression...")
    class_weights = {0: 1, 1: 8}  # 8x weight for high-risk
    lr_weighted = LogisticRegression(
        random_state=RANDOM_STATE, 
        class_weight=class_weights,
        C=0.1,  # More regularization
        max_iter=1000
    )
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    results['Weighted_LR'] = evaluate_high_risk_focused_model(
        X_train_scaled, X_test_scaled, y_train, y_test, lr_weighted, "Weighted Logistic Regression"
    )
    
    # 2. Recall-Optimized Random Forest
    print("Testing Recall-Optimized Random Forest...")
    rf_recall = RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=1,
        class_weight={0: 1, 1: 6},  # 6x weight for high-risk
        random_state=RANDOM_STATE
    )
    
    results['Recall_RF'] = evaluate_high_risk_focused_model(
        X_train, X_test, y_train, y_test, rf_recall, "Recall-Optimized Random Forest"
    )
    
    # 3. Gradient Boosting with Custom Loss
    print("Testing Gradient Boosting...")
    gb_custom = GradientBoostingClassifier(
        n_estimators=150,
        learning_rate=0.05,  # Slower learning
        max_depth=4,
        subsample=0.8,
        random_state=RANDOM_STATE
    )
    
    results['Custom_GB'] = evaluate_high_risk_focused_model(
        X_train, X_test, y_train, y_test, gb_custom, "Custom Gradient Boosting"
    )
    
    # 4. SVM with Custom Weights
    print("Testing Weighted SVM...")
    svm_weighted = SVC(
        kernel='rbf',
        C=1.0,
        gamma='scale',
        class_weight={0: 1, 1: 7},  # 7x weight for high-risk
        probability=True,
        random_state=RANDOM_STATE
    )
    
    results['Weighted_SVM'] = evaluate_high_risk_focused_model(
        X_train_scaled, X_test_scaled, y_train, y_test, svm_weighted, "Weighted SVM"
    )
    
    return results, X_train, X_test, y_train, y_test

# Test on original data
print("TESTING ON ORIGINAL IMBALANCED DATA:")
original_results, X_tr, X_te, y_tr, y_te = test_optimized_algorithms(X, y)

# Test on balanced datasets
balanced_results = {}
for strategy, (X_bal, y_bal) in balanced_datasets.items():
    if len(X_bal) > len(X) * 1.1:  # Only if we actually added samples
        print(f"\nTESTING ON {strategy.upper()} BALANCED DATA:")
        strategy_results, _, _, _, _ = test_optimized_algorithms(X_bal, y_bal)
        balanced_results[strategy] = strategy_results

# =====================================================
# STEP 4: THRESHOLD OPTIMIZATION FOR HIGH RECALL
# =====================================================

print(f"\n4. THRESHOLD OPTIMIZATION FOR HIGH RECALL")
print("=" * 50)

def optimize_threshold_for_recall(y_true, y_pred_proba, min_recall=0.80):
    """Find threshold that maximizes precision while maintaining minimum recall"""
    
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred_proba)
    
    # Find thresholds that meet minimum recall requirement
    valid_thresholds = thresholds[recall[:-1] >= min_recall]
    valid_precision = precision[:-1][recall[:-1] >= min_recall]
    valid_recall = recall[:-1][recall[:-1] >= min_recall]
    
    if len(valid_thresholds) == 0:
        # If no threshold meets min_recall, find best available
        best_idx = np.argmax(recall[:-1])
        return thresholds[best_idx], precision[best_idx], recall[best_idx]
    
    # Among valid thresholds, pick the one with highest precision
    best_idx = np.argmax(valid_precision)
    
    return valid_thresholds[best_idx], valid_precision[best_idx], valid_recall[best_idx]

# Find best model from original results
best_original_model = max(original_results.items(), 
                         key=lambda x: x[1]['high_risk_recall'])
best_name, best_result = best_original_model

print(f"Optimizing threshold for: {best_name}")
print(f"Original performance: Recall={best_result['high_risk_recall']:.3f}, "
      f"Precision={best_result['high_risk_precision']:.3f}")

# Test different recall targets
recall_targets = [0.70, 0.75, 0.80, 0.85]
threshold_results = {}

for target_recall in recall_targets:
    try:
        opt_threshold, opt_precision, achieved_recall = optimize_threshold_for_recall(
            y_te, best_result['y_pred_proba'], target_recall
        )
        
        # Apply this threshold
        y_pred_optimized = (best_result['y_pred_proba'] >= opt_threshold).astype(int)
        
        # Get detailed metrics
        opt_report = classification_report(
            y_te, y_pred_optimized, 
            target_names=['Low Risk', 'High Risk'], 
            output_dict=True, zero_division=0
        )
        
        threshold_results[target_recall] = {
            'threshold': opt_threshold,
            'precision': opt_precision,
            'recall': achieved_recall,
            'f1': opt_report['High Risk']['f1-score'],
            'accuracy': opt_report['accuracy']
        }
        
        print(f"Target Recall {target_recall:.2f}: Threshold={opt_threshold:.3f}, "
              f"Achieved Recall={achieved_recall:.3f}, Precision={opt_precision:.3f}")
        
    except Exception as e:
        print(f"Failed for target recall {target_recall}: {e}")

# =====================================================
# STEP 5: COMPARISON AND RECOMMENDATIONS
# =====================================================

print(f"\n5. PERFORMANCE COMPARISON AND RECOMMENDATIONS")
print("=" * 50)

# Collect all results
all_improvement_results = {}

# Original optimized models
for name, result in original_results.items():
    all_improvement_results[f"Optimized_{name}"] = result

# Balanced dataset results
for strategy, strategy_results in balanced_results.items():
    for name, result in strategy_results.items():
        all_improvement_results[f"{strategy}_{name}"] = result

# Sort by high-risk F1 score
sorted_improvements = sorted(
    all_improvement_results.items(), 
    key=lambda x: x[1]['high_risk_f1'], 
    reverse=True
)

print("HIGH-RISK CLASS PERFORMANCE RANKING:")
print("-" * 80)
print(f"{'Rank':<4} {'Model':<25} {'Precision':<10} {'Recall':<8} {'F1':<8} {'AUC':<8}")
print("-" * 80)

for i, (name, result) in enumerate(sorted_improvements[:10]):
    precision = result['high_risk_precision']
    recall = result['high_risk_recall']  
    f1 = result['high_risk_f1']
    auc = result.get('auc', 0)
    
    print(f"{i+1:<4} {name[:24]:<25} {precision:<10.3f} {recall:<8.3f} {f1:<8.3f} {auc:<8.3f}")

# Best model analysis
if sorted_improvements:
    best_name, best_result = sorted_improvements[0]
    
    print(f"\n🏆 BEST HIGH-RISK PERFORMANCE: {best_name}")
    print(f"   High-Risk Precision: {best_result['high_risk_precision']:.3f}")
    print(f"   High-Risk Recall:    {best_result['high_risk_recall']:.3f}")
    print(f"   High-Risk F1:        {best_result['high_risk_f1']:.3f}")
    
    if best_result['high_risk_recall'] >= 0.75:
        print(f"   ✅ EXCELLENT: Catches 75%+ of high-risk patients")
    elif best_result['high_risk_recall'] >= 0.65:
        print(f"   ✅ GOOD: Catches 65%+ of high-risk patients")
    else:
        print(f"   ⚠️ MODERATE: Still missing many high-risk patients")
    
    if best_result['high_risk_precision'] >= 0.70:
        print(f"   ✅ EXCELLENT: Low false alarm rate")
    elif best_result['high_risk_precision'] >= 0.60:
        print(f"   ✅ GOOD: Reasonable false alarm rate")
    else:
        print(f"   ⚠️ MODERATE: Higher false alarm rate")

# =====================================================
# SAVE BEST MODEL FOR FUTURE USE
# =====================================================

print(f"\n📁 SAVING BEST MODEL FOR FUTURE USE")
print("=" * 50)

# Create model package to save
model_package = {
    'model_name': best_name,
    'model_object': None,
    'scaler': None,
    'feature_names': feature_cols,
    'performance_metrics': {
        'high_risk_precision': best_result['high_risk_precision'],
        'high_risk_recall': best_result['high_risk_recall'],
        'high_risk_f1': best_result['high_risk_f1'],
        'auc': best_result.get('auc', 0),
        'accuracy': best_result.get('classification_report', {}).get('accuracy', 0)
    },
    'model_type': None,
    'requires_scaling': False,
    'random_state': RANDOM_STATE,
    'training_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'class_distribution': {
        'low_risk_count': low_risk_count,
        'high_risk_count': high_risk_count,
        'imbalance_ratio': imbalance_ratio
    }
}

# Determine which specific model to save based on best_name
if best_name == "Ensemble":
    print("⚠️ Note: Ensemble model saving requires special handling")
    print("   Saving ensemble metadata and component model references")
    model_package['model_type'] = 'ensemble'
    model_package['ensemble_info'] = {
        'threshold': ensemble_result['threshold'],
        'component_models': list(original_results.keys())
    }
    # Save ensemble metadata only
    # model_filename = f"best_high_risk_model_ensemble_rs{RANDOM_STATE}.pkl"
    model_filename = f"patient_predictor_model.pkl"
    
else:
    # Find and save the actual trained model
    model_found = False
    
    # Check original results first
    for orig_name, orig_result in original_results.items():
        if best_name == f"Optimized_{orig_name}":
            model_package['model_object'] = orig_result['model']
            model_package['model_type'] = orig_name
            
            # Check if this model required scaling
            if orig_name == "Weighted_LR" or orig_name == "Weighted_SVM":
                model_package['requires_scaling'] = True
                # Save the scaler used
                scaler = StandardScaler()
                scaler.fit(X_tr)  # Refit scaler on training data
                model_package['scaler'] = scaler
                print(f"   ✅ Scaling required - StandardScaler saved")
            
            model_found = True
            break
    
    # Check balanced dataset results if not found in original
    if not model_found:
        for strategy, strategy_results in balanced_results.items():
            for strat_name, strat_result in strategy_results.items():
                if best_name == f"{strategy}_{strat_name}":
                    model_package['model_object'] = strat_result['model']
                    model_package['model_type'] = f"{strategy}_{strat_name}"
                    
                    # Check if this model required scaling
                    if strat_name == "Weighted_LR" or strat_name == "Weighted_SVM":
                        model_package['requires_scaling'] = True
                        # Get the balanced dataset for this strategy
                        X_bal_strat, y_bal_strat = balanced_datasets[strategy]
                        X_train_bal, _, _, _ = train_test_split(
                            X_bal_strat, y_bal_strat, test_size=0.3, 
                            random_state=RANDOM_STATE, stratify=y_bal_strat
                        )
                        scaler = StandardScaler()
                        scaler.fit(X_train_bal)  # Fit on balanced training data
                        model_package['scaler'] = scaler
                        print(f"   ✅ Scaling required - StandardScaler saved (trained on {strategy} data)")
                    
                    model_found = True
                    break
            if model_found:
                break
    
    model_filename = f"patient_predictor_model.pkl"

# Save the model package
try:
    joblib.dump(model_package, model_filename)
    print(f"✅ Model saved successfully: {model_filename}")
    
except Exception as e:
    print(f"❌ Error saving model: {e}")

print(f"\n📋 MODEL PACKAGE CONTENTS:")
print(f"   • Model Type: {model_package['model_type']}")
print(f"   • Requires Scaling: {model_package['requires_scaling']}")
print(f"   • Features Used: {len(model_package['feature_names'])} features")
print(f"   • Performance: F1={model_package['performance_metrics']['high_risk_f1']:.3f}")
print(f"   • Random State: {model_package['random_state']}")

print(f"\n" + "="*60)
print(f"HIGH-RISK CLASS IMPROVEMENT ANALYSIS COMPLETE!")
print(f"="*60)


TARGETED HIGH-RISK CLASS PERFORMANCE IMPROVEMENT
Focus: Boosting Precision, Recall, and F1 for High-Risk Nutritional Patients

1. ANALYZING CLASS IMBALANCE PROBLEM
Loaded data: (179, 86)
Class Distribution Analysis:
  Low Risk (0):  116 samples (64.8%)
  High Risk (1): 63 samples (35.2%)
  Imbalance Ratio: 1.8:1 (Low:High)
  MILD IMBALANCE - Basic class weighting sufficient

2. STRATEGIC OVERSAMPLING WITH SMART VARIATION
Original: 116 low-risk, 63 high-risk
Creating 63 synthetic high-risk samples...
Balanced: 116 low-risk, 126 high-risk
Original: 116 low-risk, 63 high-risk
No oversampling needed for moderate strategy
Original: 116 low-risk, 63 high-risk
Creating 14 synthetic high-risk samples...
Balanced: 116 low-risk, 77 high-risk

3. HIGH-RECALL OPTIMIZED MODELS
TESTING ON ORIGINAL IMBALANCED DATA:
Testing High-Weight Logistic Regression...

Weighted Logistic Regression Results:
  AUC: 0.648
  High-Risk Precision: 0.394
  High-Risk Recall:    0.684
  High-Risk F1:        0.500
Testi